In [2]:
import pandas as pd
import os
import time
from dotenv import load_dotenv
from openai import OpenAI

# === Load API key ===
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# === Config ===
TASK_CSV = "task_list.csv"
TEMP = 0.2  
OUTPUT_CSV = f"{TEMP}_rec_RQ1.csv"  

# === Load entire task list ===
df = pd.read_csv(TASK_CSV)
df = df.reset_index(drop=True)

# Prepare tasks and sources
tasks = df["description"].dropna().tolist()
sources = df["source"].tolist()

# === OpenAI Client ===
client = OpenAI(api_key=api_key)

# === Store all results ===
results = []

# === 5 Rounds of recommendations ===
for run_id in range(1, 6):
    print(f"\n Starting run {run_id}/5 with temperature={TEMP}...")

    for i, (task, source) in enumerate(zip(tasks, sources)):
        print(f" Run {run_id} | Task {i+1}/{len(tasks)} ({source.upper()}): {task}")

        # Prompt for GPT
        prompt = (
            "You are a Javascript backend developer.\n"
            "Recommend an npm library based on the given task.\n"
            "Return only the library name.\n\n"
            f"Task: {task}\n"
            "library:\n"
        )

        try:
            start_time = time.time()
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=TEMP,
                timeout=60,
            )
            duration = time.time() - start_time
            answer = response.choices[0].message.content.strip()
            print(f"  Recommended: {answer} (in {duration:.1f}s)")
        except Exception as e:
            answer = "Error"
            print(" GPT error:", e)

        # Append to results
        results.append({
            "Run": run_id,
            "Task_Index": i,
            "Source": source,
            "Description": task,
            "Temperature": TEMP,
            "Recommended_Library": answer
        })

        time.sleep(1)  # Optional: be kind to the API

# === Save all results ===
output_df = pd.DataFrame(results)
output_df.to_csv(OUTPUT_CSV, index=False)
print(f"\n All results saved to: {OUTPUT_CSV}")



🚀 Starting run 1/5 with temperature=0.2...
🔍 Run 1 | Task 1/1459 (TOP): Node.js body parsing middleware
✅ Recommended: body-parser (in 0.6s)
🔍 Run 1 | Task 2/1459 (TOP): Add callbacks to requests in flight to avoid async duplication
✅ Recommended: p-limit (in 1.0s)
🔍 Run 1 | Task 3/1459 (TOP): The ultimate javascript content-type utility.
✅ Recommended: mime-types (in 1.0s)
🔍 Run 1 | Task 4/1459 (TOP): Media Type Database
✅ Recommended: mime-db (in 0.6s)
🔍 Run 1 | Task 5/1459 (TOP): Regular expression for matching ANSI escape codes
✅ Recommended: ansi-regex (in 1.0s)
🔍 Run 1 | Task 6/1459 (TOP): A JSON with CSS color names
✅ Recommended: color-name-list (in 3.0s)
🔍 Run 1 | Task 7/1459 (TOP): A querystring parser and serializer with nesting support
✅ Recommended: qs (in 0.5s)
🔍 Run 1 | Task 8/1459 (TOP): Safer Node.js Buffer API
✅ Recommended: safe-buffer (in 0.5s)
🔍 Run 1 | Task 9/1459 (TOP): Port of the OpenBSD `bcrypt_pbkdf` function to pure Javascript
✅ Recommended: bcryptjs (in 1.